In [1]:
#Instlar dependencias. !Muy importante!: Reiniciar entorno de ejecución despues de hacer las instalaciones si ejecutas en colab.
!pip install numpy==1.26.1
!pip install torch
!pip install mediapipe

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn


# Extracción caracteristicas de un video

In [3]:
# Funciones
###Face####
# Function to calculate distance between landmarks
def distance(point1, point2):
    return np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

# Function to estimate head pose angles
def get_head_pose(face_landmarks, image_width, image_height):
    # 3D model points (from MediaPipe documentation)
    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (0.0, -330.0, -65.0),        # Chin
        (-225.0, 170.0, -135.0),     # Left eye left corner
        (225.0, 170.0, -135.0),      # Right eye right corner
        (-150.0, -150.0, -125.0),    # Left Mouth corner
        (150.0, -150.0, -125.0)      # Right mouth corner
    ])

    # 2D image points (from detected landmarks)
    image_points = np.array([
        (face_landmarks.landmark[1].x * image_width, face_landmarks.landmark[1].y * image_height),
        (face_landmarks.landmark[152].x * image_width, face_landmarks.landmark[152].y * image_height),
        (face_landmarks.landmark[33].x * image_width, face_landmarks.landmark[33].y * image_height),
        (face_landmarks.landmark[263].x * image_width, face_landmarks.landmark[263].y * image_height),
        (face_landmarks.landmark[61].x * image_width, face_landmarks.landmark[61].y * image_height),
        (face_landmarks.landmark[291].x * image_width, face_landmarks.landmark[291].y * image_height)
    ], dtype="double")

    # Camera internals
    focal_length = image_width
    center = (image_width / 2, image_height / 2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype="double")

    # Solve for head pose
    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points,
                                                                  camera_matrix, dist_coeffs,
                                                                  flags=cv2.SOLVEPNP_ITERATIVE)

    # Get rotation angles (in degrees)
    rotation_mat, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = cv2.hconcat((rotation_mat, translation_vector))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    pitch, yaw, roll = euler_angles.flatten()

    return pitch, yaw, roll

###Body###

def calculate_bone_orientation(landmark1, landmark2):
    """Calcula la orientación de un hueso en los planos ZX, XY, YZ."""
    v = np.array([landmark2.x - landmark1.x, landmark2.y - landmark1.y, landmark2.z - landmark1.z])

    # Orientación en el plano ZX
    zx_angle = np.degrees(np.arctan2(v[2], v[0]))

    # Orientación en el plano XY
    xy_angle = np.degrees(np.arctan2(v[1], v[0]))

    # Orientación en el plano YZ
    yz_angle = np.degrees(np.arctan2(v[2], v[1]))

    return zx_angle, xy_angle, yz_angle

def calculate_angle(landmark1, landmark2, landmark3):
    """Calcula el ángulo entre tres puntos de referencia."""
    v1 = np.array([landmark2.x - landmark1.x, landmark2.y - landmark1.y, landmark2.z - landmark1.z])
    v2 = np.array([landmark3.x - landmark2.x, landmark3.y - landmark2.y, landmark3.z - landmark2.z])

    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)

    return np.degrees(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

def calculate_bone_orientation(landmark1, landmark2):
    """Calcula la orientación de un hueso en los planos ZX, XY, YZ."""
    v = np.array([landmark2.x - landmark1.x, landmark2.y - landmark1.y, landmark2.z - landmark1.z])

    # Orientación en el plano ZX
    zx_angle = np.degrees(np.arctan2(v[2], v[0]))

    # Orientación en el plano XY
    xy_angle = np.degrees(np.arctan2(v[1], v[0]))

    # Orientación en el plano YZ
    yz_angle = np.degrees(np.arctan2(v[2], v[1]))

    return zx_angle, xy_angle, yz_angle

def calculate_midpoint(landmark1, landmark2):
    return mp.solutions.pose.PoseLandmark(
        x=(landmark1.x + landmark2.x) / 2,
        y=(landmark1.y + landmark2.y) / 2,
        z=(landmark1.z + landmark2.z) / 2,
    )

def calculate_midpoint(landmark1, landmark2):
    return mp.solutions.pose.PoseLandmark(
        x=(landmark1.x + landmark2.x) / 2,
        y=(landmark1.y + landmark2.y) / 2,
        z=(landmark1.z + landmark2.z) / 2,
    )
def calculate_stdv(landmark1, landmark2):
    """Calcula la desviación estándar de dos landmarks."""
    x_values = [landmark1.x, landmark2.x]
    y_values = [landmark1.y, landmark2.y]
    z_values = [landmark1.z, landmark2.z]

    x_stdv = np.std(x_values)
    y_stdv = np.std(y_values)
    z_stdv = np.std(z_values)

    return mp.solutions.pose.PoseLandmark(x=x_stdv, y=y_stdv, z=z_stdv)



In [4]:
###Extractores de features###

def extract_facial_features(face_landmarks, image_width, image_height):
    """Extrae las features faciales de los landmarks."""

    if face_landmarks:
          pitch, yaw, roll = get_head_pose(face_landmarks, image_width, image_height)
          mouth_open = distance(face_landmarks.landmark[13], face_landmarks.landmark[14])
          left_eye_closed = distance(face_landmarks.landmark[159], face_landmarks.landmark[145]) < 0.02
          right_eye_closed = distance(face_landmarks.landmark[386], face_landmarks.landmark[374]) < 0.02
          left_eyebrow_lowered = face_landmarks.landmark[105].y > face_landmarks.landmark[107].y
          left_eyebrow_raised = face_landmarks.landmark[105].y < face_landmarks.landmark[107].y
          right_eyebrow_lowered = face_landmarks.landmark[334].y > face_landmarks.landmark[336].y
          right_eyebrow_raised = face_landmarks.landmark[334].y < face_landmarks.landmark[336].y
          left_eye_center_x = (face_landmarks.landmark[159].x + face_landmarks.landmark[145].x) / 2
          right_eye_center_x = (face_landmarks.landmark[386].x + face_landmarks.landmark[374].x) / 2
          gaze_direction_forward = abs(left_eye_center_x - right_eye_center_x) < 0.05
          gaze_direction_left = left_eye_center_x < right_eye_center_x
          gaze_direction_right = left_eye_center_x > right_eye_center_x
          au01_inner_brow_raiser = face_landmarks.landmark[66].y - face_landmarks.landmark[27].y
          au02_outer_brow_raiser = face_landmarks.landmark[107].y - face_landmarks.landmark[52].y
          au04_brow_lowerer = face_landmarks.landmark[52].y - face_landmarks.landmark[107].y
          au05_upper_lid_raiser = distance(face_landmarks.landmark[159], face_landmarks.landmark[145])
          au06_cheek_raiser = distance(face_landmarks.landmark[127], face_landmarks.landmark[234])
          au07_lid_tightener = distance(face_landmarks.landmark[33], face_landmarks.landmark[133])
          au09_nose_wrinkler = distance(face_landmarks.landmark[1], face_landmarks.landmark[5])
          au10_upper_lip_raiser = face_landmarks.landmark[13].y - face_landmarks.landmark[0].y
          au12_lip_corner_puller = distance(face_landmarks.landmark[78], face_landmarks.landmark[308])
          au14_dimpler = distance(face_landmarks.landmark[61], face_landmarks.landmark[291])
          au15_lip_corner_depressor = face_landmarks.landmark[0].y - face_landmarks.landmark[14].y
          au17_chin_raiser = distance(face_landmarks.landmark[152], face_landmarks.landmark[176])
          au20_lip_stretcher = distance(face_landmarks.landmark[57], face_landmarks.landmark[287])
          au23_lip_tightener = distance(face_landmarks.landmark[13], face_landmarks.landmark[14])
          au24_lip_pressor = distance(face_landmarks.landmark[0], face_landmarks.landmark[17])
          au25_lips_part = distance(face_landmarks.landmark[13], face_landmarks.landmark[14])
          au26_jaw_drop = distance(face_landmarks.landmark[13], face_landmarks.landmark[152])
          au27_mouth_stretch = distance(face_landmarks.landmark[61], face_landmarks.landmark[291])
          au43_eyes_closed = left_eye_closed and right_eye_closed
          features = {
              'SyHeadOrientation': pitch,
              'SxHeadOrientation': yaw,
              'SzHeadOrientation': roll,
              'SmouthOpen': mouth_open,
              'SleftEyeClosed': left_eye_closed,
              'SrightEyeClosed': right_eye_closed,
              'SleftEyebrowLowered': left_eyebrow_lowered,
              'SleftEyebrowRaised': left_eyebrow_raised,
              'SrightEyebrowLowered': right_eyebrow_lowered,
              'SrightEyebrowRaised': right_eyebrow_raised,
              'SgazeDirectionForward': gaze_direction_forward,
              'SgazeDirectionLeft': gaze_direction_left,
              'SgazeDirectionRight': gaze_direction_right,
              'SAu01_InnerBrowRaiser': au01_inner_brow_raiser,
              'SAu02_OuterBrowRaiser': au02_outer_brow_raiser,
              'SAu04_BrowLowerer': au04_brow_lowerer,
              'SAu05_UpperLidRaiser': au05_upper_lid_raiser,
              'SAu06_CheekRaiser': au06_cheek_raiser,
              'SAu07_LidTightener': au07_lid_tightener,
              'SAu09_NoseWrinkler': au09_nose_wrinkler,
              'SAu10_UpperLipRaiser': au10_upper_lip_raiser,
              'SAu12_LipCornerPuller': au12_lip_corner_puller,
              'SAu14_Dimpler': au14_dimpler,
              'SAu15_LipCornerDepressor': au15_lip_corner_depressor,
              'SAu17_ChinRaiser': au17_chin_raiser,
              'SAu20_LipStretcher': au20_lip_stretcher,
              'SAu23_LipTightener': au23_lip_tightener,
              'SAu24_LipPressor': au24_lip_pressor,
              'SAu25_LipsPart': au25_lips_part,
              'SAu26_JawDrop': au26_jaw_drop,
              'SAu27_MouthStretch': au27_mouth_stretch,
              'SAu43_EyesClosed': au43_eyes_closed
          }

    return features


def extract_body_features(landmarks):
    """Extrae las características del cuerpo a partir de los puntos de referencia."""
    features = []

    # Distancia
    features.append(landmarks[mp_pose.PoseLandmark.NOSE].z)  # avgDepth (aproximación)

    # Ángulos articulares
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP], landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]))  # leftShoulderAngle
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]))  # rightShoulderAngle
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_HIP], landmarks[mp_pose.PoseLandmark.RIGHT_HIP]))  # leanAngle (aproximación)
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP], landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])) # HipCenter_Spine-Spine_ShoulderCenter
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.NOSE])) # Spine_ShoulderCenter-ShoulderCenter_Head
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])) # Spine_ShoulderCenter-ShoulderCenter_ShoulderLeft
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW])) # Spine_ShoulderCenter-ShoulderCenter_ShoulderRight
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW], landmarks[mp_pose.PoseLandmark.LEFT_WRIST])) # ShoulderCenter_ShoulderLeft-ShoulderLeft_ElbowLeft
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW], landmarks[mp_pose.PoseLandmark.LEFT_WRIST], landmarks[mp_pose.PoseLandmark.LEFT_PINKY])) # ShoulderLeft_ElbowLeft-ElbowLeft_WristLeft
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])) # ShoulderCenter_ShoulderRight-ShoulderRight_ElbowRight
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_WRIST], landmarks[mp_pose.PoseLandmark.LEFT_PINKY], landmarks[mp_pose.PoseLandmark.LEFT_INDEX])) # ElbowLeft_WristLeft-WristLeft_HandLeft (aproximación)
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])) # ShoulderCenter_ShoulderRight-ShoulderRight_ElbowRight
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST], landmarks[mp_pose.PoseLandmark.RIGHT_PINKY])) # ShoulderRight_ElbowRight-ElbowRight_WristRight
    features.append(calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST], landmarks[mp_pose.PoseLandmark.RIGHT_PINKY], landmarks[mp_pose.PoseLandmark.RIGHT_INDEX])) # ElbowRight_WristRight-WristRight_HandRight (aproximación)


    # Orientaciones óseas
    bones = [
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER),  # HipCenter_Spine
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),  # Spine_ShoulderCenter
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.NOSE),  # ShoulderCenter_Head
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_SHOULDER),  # ShoulderCenter_ShoulderLeft
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),  # ShoulderCenter_ShoulderRight
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),  # ShoulderLeft_ElbowLeft
        (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),  # ElbowLeft_WristLeft
        (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_PINKY),  # WristLeft_HandLeft
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),  # ShoulderRight_ElbowRight
        (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),  # ElbowRight_WristRight
        (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_PINKY),  # WristRight_HandRight
    ]
    for landmark1_index, landmark2_index in bones:
        zx_angle, xy_angle, yz_angle = calculate_bone_orientation(landmarks[landmark1_index], landmarks[landmark2_index])
        features.extend([zx_angle, xy_angle, yz_angle])

    return features

In [5]:
###Extrator total de features###

def extract_all_features(image, face_mesh, pose):
    """Extrae las features faciales y corporales de una imagen."""

    # Convertir la imagen a RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Procesar la imagen con los extractores de features
    results_face_mesh = face_mesh.process(image_rgb)
    results_pose = pose.process(image_rgb)

    # Inicializar las features
    all_features = {}

    if results_face_mesh.multi_face_landmarks:
        face_landmarks = results_face_mesh.multi_face_landmarks[0]
        facial_features = extract_facial_features(face_landmarks, image.shape[1], image.shape[0])
        all_features.update(facial_features)

    # Extraer features corporales
    if results_pose.pose_landmarks:
        body_landmarks = results_pose.pose_landmarks.landmark
        body_posture_features = extract_body_features(body_landmarks)  # Usar la función extract_features del segundo extractor
        # Agregar las features corporales al diccionario all_features
        all_features.update({'body_' + k: v for k, v in zip(['avgDepth', 'leftShoulderAngle', 'rightShoulderAngle', 'leanAngle', 'HipCenter_SpineSpine_ShoulderCenter', 'Spine_ShoulderCenterShoulderCenter_Head', 'Spine_ShoulderCenterShoulderCenter_ShoulderLeft', 'Spine_ShoulderCenterShoulderCenter_ShoulderRight', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeft', 'ShoulderLeft_ElbowLeftElbowLeft_WristLeft', 'ShoulderCenter_ShoulderRightShoulderRight_ElbowRight', 'ElbowLeft_WristLeftWristLeft_HandLeft', 'ShoulderCenter_ShoulderRightShoulderRight_ElbowRight', 'ShoulderRight_ElbowRightElbowRight_WristRight', 'ElbowRight_WristRightWristRight_HandRight', 'HipCenter_SpinePlaneZXAxisX', 'HipCenter_SpinePlaneXYAxisY', 'HipCenter_SpinePlaneYZAxisZ', 'Spine_ShoulderCenterPlaneZXAxisX', 'Spine_ShoulderCenterPlaneXYAxisY', 'Spine_ShoulderCenterPlaneYZAxisZ', 'ShoulderCenter_HeadPlaneZXAxisX', 'ShoulderCenter_HeadPlaneXYAxisY', 'ShoulderCenter_HeadPlaneYZAxisZ', 'ShoulderCenter_ShoulderLeftPlaneZXAxisX', 'ShoulderCenter_ShoulderLeftPlaneXYAxisY', 'ShoulderCenter_ShoulderLeftPlaneYZAxisZavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg', 'ShoulderCenter_ShoulderRightPlaneXYAxisY', 'ShoulderCenter_ShoulderRightPlaneYZAxisZ', 'ShoulderLeft_ElbowLeftPlaneZXAxisX', 'ShoulderLeft_ElbowLeftPlaneXYAxisY', 'ShoulderLeft_ElbowLeftPlaneYZAxisZ', 'ElbowLeft_WristLeftPlaneZXAxisX', 'ElbowLeft_WristLeftPlaneXYAxisY', 'ElbowLeft_WristLeftPlaneYZAxisZ', 'WristLeft_HandLeftPlaneZXAxisX', 'WristLeft_HandLeftPlaneXYAxisY', 'WristLeft_HandLeftPlaneYZAxisZ', 'ShoulderRight_ElbowRightPlaneZXAxisX', 'ShoulderRight_ElbowRightPlaneXYAxisY', 'ShoulderRight_ElbowRightPlaneYZAxisZ', 'ElbowRight_WristRightPlaneZXAxisX', 'ElbowRight_WristRightPlaneXYAxisY', 'ElbowRight_WristRightPlaneYZAxisZ', 'WristRight_HandRightPlaneZXAxisX', 'WristRight_HandRightPlaneXYAxisY', 'WristRight_HandRightKinectZAxis'], body_posture_features)})  # Agregar prefijo 'body_' a las claves
        all_features = {k: (1 if v is True else 0) if isinstance(v, bool) else v for k, v in all_features.items()}
    return all_features

1. Importar las bibliotecas necesarias:

In [6]:
#import cv2
#import mediapipe as mp
#import numpy as np

2. Inicializar los extractores de features:

In [7]:
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

pose = mp_pose.Pose(
    static_image_mode=False,  # Para video, se debe establecer en False
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [43]:
# Para imagen:
#image = cv2.imread("Foto.jpg")  # Reemplaza con la ruta a tu imagen

# Para video:
cap = cv2.VideoCapture("Video.mp4")  # Reemplaza con la ruta a tu video
FPS = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second

In [44]:
# Para imagen:
#all_features = extract_all_features(image)
#print(all_features)
# Para video:
all_frames_features = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    all_features = extract_all_features(frame, face_mesh, pose)
    all_frames_features.append(all_features)

cap.release()  # Release the video capture object

features_df = pd.DataFrame(all_frames_features)

In [46]:
# Calculate mean and std for each minute
minute_data = []
for minute in range(int(len(features_df) / (FPS * 1)) + 1):  # Iterate over minutes
    start_frame = int(minute * FPS * 1)  # Convert to int
    end_frame = int((minute + 1) * FPS * 1)  # Convert to int
    minute_df = features_df.iloc[start_frame:end_frame]  # Data for the current minute


    body_features = minute_df.filter(regex='^body_') # Select only body features
    facial_features = minute_df.drop(body_features.columns, axis=1)  # Select only facial features


    facial_mean_values = facial_features.mean()
    body_mean_values = body_features.mean()
    body_std_values = body_features.std()


    minute_data.append({
        'minute': minute + 1,
        **facial_mean_values.to_dict(),   # Unpack facial mean values
        **{k + 'avg': v for k, v in body_mean_values.to_dict().items()}, # Unpack body mean values
        **{k + 'stdv': v for k, v in body_std_values.to_dict().items()}  # Unpack body std values

    })

minute_features_df = pd.DataFrame(minute_data)

minute_features_df = minute_features_df.rename(columns={col: col.replace('body_', '') for col in minute_features_df.columns if col.startswith('body_')})

In [47]:
minute_features_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
minute,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000
SyHeadOrientation,-172.129726,-173.225098,-178.026604,-174.640301,-168.659149,-170.586365,-174.483985,-172.684064,-167.908509,-176.581975,-135.453145,-174.400291,-167.926959,-173.327197,-92.215737,-174.904382
SxHeadOrientation,3.850241,30.351498,33.511478,35.943309,33.344594,33.452378,36.774313,31.665927,27.919911,26.930147,29.105645,23.927778,-12.620109,-10.929159,-11.916764,-11.668979
SzHeadOrientation,6.153457,-14.463503,-14.879800,-13.251258,6.588980,-16.933446,-6.402111,108.891119,11.138681,-8.174218,-7.782848,14.890762,-84.112543,-31.186571,-16.712496,-86.715898
SmouthOpen,0.005538,0.014877,0.013378,0.009389,0.003264,0.004980,0.011597,0.006918,0.016456,0.014022,0.007857,0.012271,0.018518,0.009340,0.024050,0.015237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ElbowRight_WristRightPlaneXYAxisYstdv,49.034782,2.963972,1.873834,1.739532,0.946669,0.462346,2.513304,2.022314,1.834750,1.554074,0.754446,2.124201,0.732516,1.258949,1.370107,0.257676
ElbowRight_WristRightPlaneYZAxisZstdv,15.288501,1.776074,1.341703,1.288292,0.962781,0.440977,6.185051,3.901215,1.613033,2.425214,2.854338,3.702279,4.236573,2.560832,2.777260,1.628835
WristRight_HandRightPlaneZXAxisXstdv,2.358049,2.699421,2.102112,1.488165,2.170880,0.845345,2.035178,2.150821,2.203507,2.008174,1.904497,2.229196,2.188228,1.935451,1.083808,1.189378
WristRight_HandRightPlaneXYAxisYstdv,49.025679,2.705370,3.010263,2.486342,2.572621,0.839149,0.280022,2.694077,2.750554,1.646216,1.562800,1.748844,1.463607,1.098983,1.024666,1.365569


# Prueba de predicción con LSTM usando features extraidas de un video

In [95]:
#import torch
#import torch.nn as nn

class StressLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=88, num_layers=2, dropout=0.4550597814102049):
        super(StressLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)  # Batch Normalization
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x.unsqueeze(1))  # (batch, seq_len=1, input_dim)
        lstm_out = self.batch_norm(lstm_out[:, -1, :])  # Última salida del LSTM con batch norm
        return self.fc(lstm_out)  # Pasar por la capa totalmente conectada

In [96]:
#import torch
modelo_LSTM = torch.load('modelo_LSTM.pkl', weights_only=False)
modelo_LSTM.eval()

StressLSTM(
  (lstm): LSTM(134, 88, num_layers=2, batch_first=True, dropout=0.455059781)
  (batch_norm): BatchNorm1d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=88, out_features=1, bias=True)
)

In [120]:
#from sklearn.preprocessing import StandardScaler
y = np.array(range(0,11)).reshape(-1, 1)
data_to_scale = minute_features_df.drop(columns=['minute'])
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_tensor = torch.tensor(scaled_data, dtype=torch.float32)
scaled_y = scaler.fit_transform(y)

In [121]:
padding_size = 134 - scaled_tensor.shape[1]  # Calculate padding size
if padding_size > 0:
    padding = torch.zeros(scaled_tensor.shape[0], padding_size, dtype=torch.float32)
    scaled_tensor = torch.cat([scaled_tensor, padding], dim=1)

In [125]:
with torch.no_grad():
    prediction = modelo_LSTM(scaled_tensor)
prediction = [round(x) for x in scaler.inverse_transform(np.array(prediction).reshape(-1, 1)).flatten()]
print("Predicción:", prediction)

Predicción: [5, 6, 5, 5, 5, 6, 5, 4, 3, 5, 5, 4, 3, 6, 4, 6]


# Prueba con xgb

In [18]:
##Entrenar modelo con los mejores hiperparámetros
#hiperparametros = {'n_estimators': 144,
#                   'max_depth': 4,
#                   'learning_rate': 0.02928947348634358,
#                   'subsample': 0.6839171426671771,
#                   'colsample_bytree': 0.7849449085327913,
#                   'reg_lambda': 1.4331398962320347,
#                   'reg_alpha': 0.2753147917266039,
#                   'min_child_weight': 5,
#                   'gamma': 0.6505640521246205}

In [126]:
import xgboost as xgb

modelo_xgb = xgb.Booster()
modelo_xgb.load_model('modelo_xgb.ubj')

In [127]:
data_to_predict = minute_features_df.drop(columns=['minute'])

In [128]:
feature_names_modelo = modelo_xgb.feature_names
feature_names_entrada = data_to_predict.columns.tolist()

features_sobrantes_en_entrada = set(feature_names_entrada) - set(feature_names_modelo)
features_faltantes_en_entrada = set(feature_names_modelo) - set(feature_names_entrada)



In [129]:
data_to_predict = data_to_predict.drop(columns=list(features_sobrantes_en_entrada))

In [131]:
#for feature in features_faltantes_en_entrada:
#  data_to_predict[feature] = 0  # Or another appropriate default value
data_to_predict = data_to_predict.assign(**{feature: 0 for feature in features_faltantes_en_entrada})

In [132]:
data_to_predict = data_to_predict[feature_names_modelo]

In [61]:
dmatrix_data = xgb.DMatrix(data_to_predict)

In [134]:
predicciones = np.round(modelo_xgb.predict(dmatrix_data),0)

In [135]:
print("Predicciones:", predicciones)

Predicciones: [4. 3. 3. 4. 3. 3. 3. 4. 4. 4. 4. 4. 3. 4. 4. 4.]


# Prueba con RF

In [64]:
import pickle

In [72]:
with open('modelo_Random_Forest.pkl', 'rb') as f:
    modelo_RF = pickle.load(f)


In [73]:
data_to_predict = minute_features_df.drop(columns=['minute'])

In [74]:
feature_names_modelo = modelo_RF.feature_names_in_

In [75]:
feature_names_entrada = data_to_predict.columns.tolist()

features_sobrantes_en_entrada = set(feature_names_entrada) - set(feature_names_modelo)
features_faltantes_en_entrada = set(feature_names_modelo) - set(feature_names_entrada)


In [76]:
data_to_predict = data_to_predict.drop(columns=list(features_sobrantes_en_entrada))

In [77]:
for feature in features_faltantes_en_entrada:
    data_to_predict[feature] = 0

In [78]:
data_to_predict = data_to_predict[feature_names_modelo]

In [79]:
predicciones = modelo_RF.predict(data_to_predict)

In [80]:
print("Predicciones:", predicciones)

Predicciones: [6 3 3 3 6 3 6 6 6 6 6 6 6 6 6 6]


# Prueba Decision Tree

In [83]:
import pickle
with open('modelo_DecisionTree.pkl', 'rb') as f:
    modelo_RF = pickle.load(f)
data_to_predict = minute_features_df.drop(columns=['minute'])
feature_names_modelo = modelo_RF.feature_names_in_
feature_names_entrada = data_to_predict.columns.tolist()

features_sobrantes_en_entrada = set(feature_names_entrada) - set(feature_names_modelo)
features_faltantes_en_entrada = set(feature_names_modelo) - set(feature_names_entrada)
data_to_predict = data_to_predict.drop(columns=list(features_sobrantes_en_entrada))
for feature in features_faltantes_en_entrada:
    data_to_predict[feature] = 0
data_to_predict = data_to_predict[feature_names_modelo]
scaler = StandardScaler()
data_to_predict = scaler.fit_transform(data_to_predict)
predicciones = modelo_RF.predict(data_to_predict)
print("Predicciones:", predicciones)

Predicciones: [ 2  2  2  2  4  1  5 10  2  3  3  5  1  1  3  3]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
